# Testing SARO

In [ ]:
import numpy as np
import saro
import xara
import matplotlib.pyplot as plt

from time import time
from scipy.linalg import sqrtm
import scipy.sparse as sparse

## Creating the KPO structure from a pupil text file

In [ ]:
thekpo = xara.KPO("test_data/pupil_model.txt")
a = thekpo.kpi.plot_pupil_and_uv(cmap="inferno")
plt.show(a)

## Defining basic parameters for the analysis

In [ ]:
wl = 1.6e-6
m2pix = xara.mas2rad( 16.7 * 128 / wl)#pscale supposed 16.7
thekpo.CWAVEL = wl

## Importing the test case data

In [ ]:
calibrator = np.load("test_data/calibrator.npy")
tarimage_single = np.load("test_data/tarimage_single.npy")
tarimage_series = np.load("test_data/tarimage_series.npy")
params_single = np.load("test_data/params_single.npy")
params_series = np.load("test_data/params_series.npy")
detpas = np.load("test_data/detpas.npy")

## Extracting the visibilities and kernel-phases

In [ ]:
cviscal = thekpo.extract_cvis_from_img(calibrator, m2pix)
kappacal = thekpo.kpi.KPM.dot(np.angle(cviscal))
cvistar_single = thekpo.extract_cvis_from_img(tarimage_single, m2pix)
kappasingle = thekpo.kpi.KPM.dot(np.angle(np.angle(cvistar_single)))
cvistar_series = np.array([thekpo.extract_cvis_from_img(tarimage_series[i], m2pix) for i in range(detpas.shape[0])])
kappa_series = thekpo.kpi.KPM.dot(cvistar_series.T).T.flatten()

## Importing the covariance matrix
There are several ways to build a covariance matrix for the data. None of them are perfect.

Here the covariance was evaluated from a cube of 10k realizations of independent phase screen and shot noise on a calibrator. It should be very close to ideal.

In [ ]:
Sigma = np.load("test_data/covk_10000_250_50_2_apod_False.npy")
Sigmacal = Sigma.copy()
assert Sigma.shape[0] == kappacal.shape[0]
plt.figure(figsize=(10,10))
plt.imshow(Sigma, cmap="inferno")

Note that this matrix is very much NOT diagonal, which shows that the kernel-phases, just like any Fourier plane observables, are heavily correlated. This is the reason we need to use the whitening transforms.

Here, we need a whitening matrix for the calibrated data, so the matrix has to be adjusted!

In [ ]:
W = sqrtm(np.linalg.inv(Sigma + Sigmacal))
Sigmas = np.array([Sigma for i in range(detpas.shape[0])])
Ws = sqrtm(np.linalg.inv(sparse.block_diag(Sigmas).toarray()))